In [ ]:
import zipfile
from lxml import etree

def get_epub_info(fname):
    ns = {
        'n':'urn:oasis:names:tc:opendocument:xmlns:container',
        'pkg':'http://www.idpf.org/2007/opf',
        'dc':'http://purl.org/dc/elements/1.1/'
    }

    # prepare to read from the .epub file
    zip = zipfile.ZipFile(fname)

    # find the contents metafile
    txt = zip.read('META-INF/container.xml')
    tree = etree.fromstring(txt)
    cfname = tree.xpath('n:rootfiles/n:rootfile/@full-path',namespaces=ns)[0]

    # grab the metadata block from the contents metafile
    cf = zip.read(cfname)
    tree = etree.fromstring(cf)
    p = tree.xpath('/pkg:package/pkg:metadata',namespaces=ns)[0]

    # repackage the data
    res = {}
    for s in ['title','language','creator','date','identifier']:
        res[s] = p.xpath('dc:%s/text()'%(s),namespaces=ns)[0]

    return res

In [2]:
import glob, os
import subprocess
from ebooklib import epub

In [10]:
path="/media/some/sophie-hhd/books/mobi-m/"
tpath="/media/some/sophie-hhd/books/epub-m/"

In [11]:


os.chdir(path)
cnt=0
listazw3=[]
for f in glob.glob("*.mobi"):
    ff=path+f
    tf=tpath+f.lower()[:-4]+"epub"
    if not os.path.isfile(tf):
        
        m=!ebook-convert "$ff" "$tf"
    #book=epub.read_epub(tf)
    #print book.get_metadata("DC", "title")
    #listazw3.append(meta)
    #cnt+=1
    #if cnt>5: break
os.chdir("/home/some")

In [7]:
9


9

In [ ]:

m

In [ ]:
book=epub.read_epub(tpath+"#3 Brie's Russian Fantasy - Red Phoenix.epub")
print book.get_metadata("DC", "identifier")
#book.set_unique_metadata('DC', 'identifier', book.uid, {'id': book.IDENTIFIER_ID})
book.set_identifier('sample123456')
book.uid

In [ ]:
not os.path.isfile(tpath+"#3 Brie's Russian Fantasy - Red Phoenix.epub") 

In [ ]:
import ebooklib

In [ ]:
mm="01 Jack's Dee-Light - Thorn, Lacey.mobi" 


In [ ]:
bashCommand = "ebook-meta '"+mm+"' >ff.txt"
print bashCommand
os.system(bashCommand)


In [ ]:

s=!ebook-meta "$listazw3[0]"

In [ ]:

!for i in $(find /media/some/sophie-hhd/books/mobi -type f -iname *.mobi) ;do m="$i" ;done

In [ ]:
lst=!ls /media/some/sophie-hhd/books/mobi

In [ ]:
!for i in $(ls /media/some/sophie-hhd/books/mobi); do ebook-meta "'$i'";done

In [ ]:
res

In [ ]:
#!/bin/sh

# ebook-rename
# by Miriam  2013-06-27,28,29
#
# uses calibre's ebook-meta to read out the data
# then sed to extract the author and title
# and renames the original file


# test to see if started from CLI or icon
#tstcli=`tty | head -c3`
#if [ "$tstcli" = "not" ]; then
#   #xmessage "`tty`"
#   xmessage "EEEK!! Don't click here!"
#   exit
#fi

# change dir to path of file
!cd media/some/sophie-hdd/books/mobi

!function show_options {
!   echo -e "\e[34musage: \e[35mebook-rename \e[34m"
!   echo -e "  Renames an ebook file to 'author - title'"
!   echo -e "  by extracting info from the ebook's metadata.\e[30m"
!}

!if [ "$1" = "" -o "$1" = "-h" ]; then
!   show_options
!else

   # I had previously over-complicated the reading of meta-data
   # by using the switch --from-opf not realising that I don't need any switches
   # in order to read meta-data. I know in retrospect this seems obvious, but
   # in my defense this doesn't seem to be mentioned in the docs at all.
!  ebook-meta "$1" >/tmp/ebook-meta-output

   # get author
   # ----------
   # send the metadata via cat
   # sed extracts the author line
   # sed extracts the author's name from that line
   # sed re-arranges the name if it has surname first
   # sed turns dots into spaces (they interfere with easy use of rox for various operations)
   # sed turns multiple spaces into one
   # sed gets rid of any leading spaces, just in case
!author=`cat /tmp/ebook-meta-output | sed -n '/^Author/p' | sed 's/.*: \(.*\)/\1/' | sed 's/^\(.*\).\[.*/\1/' | sed 's/\(.*\), \(.*\)/\2 \1/' | sed 's/\./ /g' | sed 's/  */ /g' | sed 's/^ *//'`

   # get title
   # ---------
   # send the metadata via cat
   # sed extracts the title line
   # sed extracts the title from that line
!title=`cat /tmp/ebook-meta-output | sed -n '/^Title   /p' | sed 's/[^:]*: \(.*\)/\1/' | sed 's|/|-|g'`

   # get filename without path
!fname="${1##*/}"
   # get filename without extension
!bname="${fname%.*}"
   # get extension
!ename="${fname##*.}"

   # rename it
!  images=`sed 's/.*-\(images\)/ (+\1)/' <<<"$bname"`
!   if [ "$images" = "$bname" ] ; then images="" ; fi
!   if [ -e "${author} - ${title}${images}.${ename}" ] ; then
!      echo "Can't rename -- \"${author} - ${title}${images}.${ename}\" already exists"
!   else
!      mv "$1" "${author} - ${title}${images}.${ename}"
!   fi

!fi

In [9]:
tpath="/media/some/sophie-hhd/books/epub-m/"
os.chdir(tpath)
cnt=0
listazw3=[]
for f in glob.glob("*.epub"):
    ff=tpath+f
    try:
        book=epub.read_epub(ff)
        title=''.join(ch for ch in book.get_metadata("DC", "title")[0][0] if ch.isalnum())
        author=''.join(ch for ch in book.get_metadata("DC","creator")[0][0] if ch.isalnum())
        '''if len(title)>20:
            title=title[:20]
        if len(author)>20:
            author=author[:20]'''
        name=title+"-"+author+".epub"
        if 'identifier' in book.metadata['http://purl.org/dc/elements/1.1/'].keys():
            book.metadata['http://purl.org/dc/elements/1.1/'].pop('identifier')
            book.set_identifier('0')
        if 'publisher' in book.metadata['http://purl.org/dc/elements/1.1/'].keys():
            book.metadata['http://purl.org/dc/elements/1.1/'].pop('publisher')
        if 'date' in book.metadata['http://purl.org/dc/elements/1.1/'].keys():
            book.metadata['http://purl.org/dc/elements/1.1/'].pop('date')
        

        if not os.path.isfile(name):
            name=name.lower()
            #print name
            epub.write_epub(path+name,book,{} )
    except epub.EpubException:
        os.remove(ff)
        print "removing bad "+ff
    #listazw3.append(meta)
    #cnt+=1
    #if cnt>5: break
os.chdir("/home/some")

KeyboardInterrupt: 